# List of Fighter Aircraft 

This is a list of military aircraft that are primarily designed for air-to-air combat and thus does not include aircraft intended for other roles where they have some secondary air-to-air capability, such as with many ground attack aircraft. 

We will be using pandas, requests and beautifulsoup to webscrape the list of Fighter Aircraft wikipedia page.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

page = requests.get('https://en.wikipedia.org/wiki/List_of_fighter_aircraft').text
soup = BeautifulSoup(page, 'html.parser')
table = soup.find('table', class_="wikitable sortable")

df = pd.read_html(str(table))
df = pd.concat(df)
print(df)

                                         Type  Country                 Class  \
0                                    ACAZ C.2  Belgium      Two-seat fighter   
1                     Adamoli-Cattani fighter    Italy                   NaN   
2                                    AD Scout       UK  Zeppelin interceptor   
3     ADA/HAL Advanced Medium Combat Aircraft    India                   NaN   
4                       AEG D.I, D.II & D.III  Germany                   NaN   
...                                       ...      ...                   ...   
1302                   Yakovlev Yak-50 (1949)     USSR                   NaN   
1303                            Yatsenko I-28     USSR                   NaN   
1304                          Yokosuka D4Y2-S    Japan         Night fighter   
1305                     Yokosuka P1Y2 Kyokko    Japan         Night fighter   
1306            Zeppelin-Lindau (Dornier) D.I  Germany                   NaN   

      Date      Status  No.          No

### Reading data in pandas

In [10]:
# Read headers
df.columns

Index(['Type', 'Country', 'Class', 'Date', 'Status', 'No.', 'Notes'], dtype='object')

### Make changes to the data

In [14]:
# Drop column
df = df.drop(columns=['Notes'])

### Filtering data

In [20]:
# Find Aircraft from the US and Operational
us_aircraft = df.loc[(df['Country'] == "US") & (df['Status'] == "Operational")]

us_aircraft

,Type,Country,Class,Date,Status,No.
196,Boeing F/A-18E/F Super Hornet,US,Carrier fighter-bomber,1995,Operational,608
472,General Dynamics F-16 Fighting Falcon,US,NaN,1974,Operational,4500
519,Grumman F-14 Tomcat,US,Carrier interceptor,1970,Operational,712
718,Lockheed Martin F-22 Raptor,US,NaN,1997,Operational,195
719,Lockheed Martin F-35 Lightning II,US,NaN,2006,Operational,800+
770,McDonnell Douglas F-15 Eagle,US,Interceptor,1972,Operational,1196
771,McDonnell Douglas F-15E Strike Eagle,US,Fighter-bomber,1986,Operational,418
772,McDonnell Douglas F/A-18 Hornet,US,Carrier fighter-bomber,1978,Operational,1480
776,McDonnell Douglas F4H/F-110/F-4 Phantom II,US,Carrier fighter-bomber,1958,Operational,5195
928,Northrop F-5,US,NaN,1959,Operational,2246


### Sorting data

In [19]:
us_aircraft.sort_values(['Date'], ascending=[1])

,Type,Country,Class,Date,Status,No.
776,McDonnell Douglas F4H/F-110/F-4 Phantom II,US,Carrier fighter-bomber,1958,Operational,5195
928,Northrop F-5,US,NaN,1959,Operational,2246
519,Grumman F-14 Tomcat,US,Carrier interceptor,1970,Operational,712
770,McDonnell Douglas F-15 Eagle,US,Interceptor,1972,Operational,1196
472,General Dynamics F-16 Fighting Falcon,US,NaN,1974,Operational,4500
772,McDonnell Douglas F/A-18 Hornet,US,Carrier fighter-bomber,1978,Operational,1480
771,McDonnell Douglas F-15E Strike Eagle,US,Fighter-bomber,1986,Operational,418
196,Boeing F/A-18E/F Super Hornet,US,Carrier fighter-bomber,1995,Operational,608
718,Lockheed Martin F-22 Raptor,US,NaN,1997,Operational,195
719,Lockheed Martin F-35 Lightning II,US,NaN,2006,Operational,800+
